# Challenge 1

In [1]:
# librarys used
import requests

In [2]:
# get instructors transaction data to identify students
instructor_address = "https://mempool.space/testnet/api/address/mvvssVimoMGdJ2fExKvH1ydjqRQLdbswQe/txs"
response = requests.get(instructor_address)
instructor_txs = response.json()


In [3]:
# a total of 50 transactions
count = 0
for tx in instructor_txs:
     count += 1
print(count)

50


In [4]:
# checkpoint one: sending money to instructors, these are the potential students
transaction_student_id = []
for tx in instructor_txs:
    prev_trans = []
    for index, receiver in enumerate(tx["vout"]):
        if receiver["scriptpubkey_address"] == "mvvssVimoMGdJ2fExKvH1ydjqRQLdbswQe":  
            transaction_student_id.append(tx["txid"])
            break

In [5]:
def check_trans(txid):
    trans_address = f"https://mempool.space/testnet/api/tx/{txid}"
    response = requests.get(trans_address)
    trans_tx = response.json()
    return trans_tx

If a transaction's last vout is Instrutor's key, it could be sending their remaining money. However, if there is transaction after sending to the instructor, then it is not by a student. If there is transaction before Instrutor, it might be student senting to another student, we should record it.

In [6]:
transaction_student_id_2 = []
transaction_student_id_notpass = []
student_potential = []
for index,tx in enumerate(transaction_student_id):
    # check the txid that was sent to instructor
    trans_tx = check_trans(tx)
    might_student = True
    findProf = False
    for i in range(len(trans_tx["vout"])-1, -1, -1):
        # if not instructor and havent find instructor, moving from back to front, it is not a student
        if trans_tx["vout"][i]["scriptpubkey_address"] != "mvvssVimoMGdJ2fExKvH1ydjqRQLdbswQe": 
            if trans_tx["vout"][i]["value"] != 0 and not findProf:
                might_student = False
                # if already find instrutor, prev transactions might be sending to students
            if might_student and trans_tx["vout"][i]["value"] != 0 and findProf:
                student_potential.append(trans_tx["vout"][i]["scriptpubkey_address"])
        if trans_tx["vout"][i]["scriptpubkey_address"] == "mvvssVimoMGdJ2fExKvH1ydjqRQLdbswQe": 
            findProf = True
        if might_student == False:
            break
    if  might_student == True:
        transaction_student_id_2.append(tx)
    else:
        transaction_student_id_notpass.append(tx)

In [7]:
def getprevAddress(tx):
    if tx["vin"][0]['is_coinbase'] == False:
        return tx["vin"][0]["prevout"]["scriptpubkey_address"]

In [8]:
student_potential = []
for tx in transaction_student_id_2:
    tx_data = check_trans(tx)
    address = getprevAddress(tx_data)
    if address != None:
        student_potential.append(address)

In [9]:
student_potential


['mkLG9f8gkxE1WRwHXSrmkbY8PJZrE6nvwT',
 'myqS6CYCpTQ1pBxuAJUXeJ9nG6posP5fk5',
 'mow22DE1bZreNCKCiYuLqbdJgcqKVpUFEz',
 'mpKLDRQ6FbPfLyn9Ltez5y2SVVywhD5UpV',
 'mtTJ8cd7ueASBGSaJgDFW2XpbEg38CHv5d',
 'mnuvBdZZVMbi8HLmzD3jhLb7ZK92HTm8yt',
 'mtq8CHWT2zNf2DbmWBAqJm1G8ATSQpbNuH',
 'miWjfZrPXtJgw5dCZwt7QteXXVisz2B76L',
 'mg16N7vFdsj5A7KcUUqtWhtuXsXgffAHbH',
 'mxXVdSsDeyju7rHmgUbhaxEnUh7WEBTmUZ',
 'miwuEN8mtZLdD6Xu2Y984hkED2okxSteR1',
 'mtLGK7Esq6osYWPGSGnU5dnwR4A81Tx6hf',
 'msFSxwS26KEu9PL2jKxPzpaWKQNSaiAYs5',
 'motL6Hw6haUuRP2m4navXdxk58VP1SPmC6',
 'mhi2WES4Po2bkGTpgD99nvT8FytFJbnRRS',
 'mfqLuta34tcfgyPEd4UTXqGTeJSFSBTC7P',
 'myan7jo4NjvJBFwMidsAspz9MiTWDsJKM1',
 'mqecNCZ8Mac3zDYtrzEXVybMizHpPHXEki',
 'n4U35yvC6sxwS5uJgb5RTzqT4ADimRnPDk',
 'n4U35yvC6sxwS5uJgb5RTzqT4ADimRnPDk',
 'n3iGRVHSUkurUZu5sUe8NZ9h7aPMbZqgqG',
 'mqZkgccnwiYTjESxxYE8jLRrUKftpd695i']

In [10]:
# get the txid from the students
transaction_final = []

for student in student_potential:
    # get these students transactions
    student_address = "https://mempool.space/testnet/api/address/mtq8CHWT2zNf2DbmWBAqJm1G8ATSQpbNuH/txs"
    response = requests.get(student_address)
    student_txs = response.json()
    for stx in student_txs:
        transaction_final.append(stx["txid"])

In [11]:
len(transaction_final)

88

In [12]:
transaction_final

['9894a2e3f58cfcfe8621328faa660ed7891aade21c4ce9a998f4a28895ebb791',
 '5366895629f1bf92cfa6e40d6005482329808da83584fec4e003196541c568de',
 'fc1dcac8020fe01dc5c612a6b9256c9a97ea061a8d132d85936ee8c480e39f25',
 '1f1e96e8a15b438a849cf2f37381b5a011db37ec934778168d0d77f65a534c0b',
 '9894a2e3f58cfcfe8621328faa660ed7891aade21c4ce9a998f4a28895ebb791',
 '5366895629f1bf92cfa6e40d6005482329808da83584fec4e003196541c568de',
 'fc1dcac8020fe01dc5c612a6b9256c9a97ea061a8d132d85936ee8c480e39f25',
 '1f1e96e8a15b438a849cf2f37381b5a011db37ec934778168d0d77f65a534c0b',
 '9894a2e3f58cfcfe8621328faa660ed7891aade21c4ce9a998f4a28895ebb791',
 '5366895629f1bf92cfa6e40d6005482329808da83584fec4e003196541c568de',
 'fc1dcac8020fe01dc5c612a6b9256c9a97ea061a8d132d85936ee8c480e39f25',
 '1f1e96e8a15b438a849cf2f37381b5a011db37ec934778168d0d77f65a534c0b',
 '9894a2e3f58cfcfe8621328faa660ed7891aade21c4ce9a998f4a28895ebb791',
 '5366895629f1bf92cfa6e40d6005482329808da83584fec4e003196541c568de',
 'fc1dcac8020fe01dc5c612a6b9256c9a